In [24]:
import pandas as pd
import ast
import torch
import json
import random
from collections import defaultdict
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import numpy as np  




with open('scrap_data.json', 'r') as file:
    data = json.load(file)
print(data.keys())

usernames_without_data = data['usernames_without_data']
usernames_with_data = data['usernames_with_data']

print(f"Total usernames with data: {len(usernames_with_data)}")

# Print the first 3 usernames and their data
for i, (username, items) in enumerate(usernames_with_data.items()):
    if i >= 3:
        break
    print(f"Username: {username}")
    print("Items:")
    for item, count in items:
        print(f"  - {item}: {count}")
        

dict_keys(['usernames_without_data', 'usernames_with_data'])
Total usernames with data: 7476
Username: !---slappdash---!
Items:
  - Rupture (band): 30
  - The Cobra Group: 22
  - Trasharama A-Go-Go: 19
  - Blood Duster: 12
  - Propagandhi: 3
  - Australian hardcore: 3
  - Brutal Truth: 2
  - Dropdead: 1
  - Wacken Open Air: 1
  - Pyramid scheme: 1
Username: %D0%90
Items:
  - List of Sundance Film Festival award winners: 60
  - List of films at the 2008 Sundance Film Festival: 59
  - Weber State University: 18
  - Ogden, Utah: 17
  - 2008 Sundance Film Festival: 14
  - List of MyNetworkTV affiliates: 13
  - KTVX: 12
  - List of films at the 2007 Sundance Film Festival: 11
  - List of ABC television affiliates (by U.S. state): 11
  - 2006 Sundance Film Festival: 11
Username: 'sed
Items:
  - The Mansion Family: 1
  - Bloody Mary (South Park): 1
  - Cartoon Wars Part I: 1
  - Cartoon Wars Part II: 1
  - The Eight I'd Really Rather You Didn'ts: 1


In [42]:
#Full batch dataset
from itertools import islice

# Get the first 100 users from usernames_with_data (without converting the whole dictionary to a list)
#first_100_users = dict(islice(usernames_with_data.items(), 100))

# Categories list, it looks like you already have this correctly defined
categories = [
    "Academic disciplines", "Business", "Communication", "Concepts", "Culture", "Economy", "Education", "Energy",
    "Engineering", "Entertainment", "Entities", "Food and drink", "Geography", "Government", "Health", "History",
    "Human behavior", "Humanities", "Information", "Internet", "Knowledge", "Language", "Law", "Life", "Lists", "Mass media",
    "Mathematics", "Military", "Nature", "People", "Philosophy", "Politics", "Religion", "Science", "Society", "Sports",
    "Technology", "Time", "Universe", "Other"
]

categoryString = "\n".join([f"{item}" for item in categories])

# Function to classify users in batches of 10
def classify_users_in_batches(usernames_with_data, batch_size=10):
    all_responses = []  # Store all responses
    
    # Loop over the users in batches of 10
    for i in range(0, len(usernames_with_data), batch_size):
        sampled_users = list(usernames_with_data.keys())[i:i + batch_size]
        
        # Step 1: Create a formatted string for the LLM input for each batch
        user_articles = []
        for username in sampled_users:
            articles = usernames_with_data[username]
            formatted_articles = [str(article) for article, _ in articles]
            user_articles.append(f"User: {username}\nArticles: {', '.join(formatted_articles)}")
        llm_input = "\n\n".join(user_articles)
        
        # Step 2: Prepare the classification prompt
        content = (
            f"Below is a list of 40 categories:\n{categoryString}\n\n"
            "You will be given a list of users along with their article names. "
            "Your task is to classify each user into one or more of the given categories based on the context and content of their article names. "
            "The response must be in the following format:\n"
            "<user_name>: category1, category2, category3, category4\n\n"
            "Instructions:\n"
            "1. Use only the provided categories.\n"
            "2. Assign at least one category to each user.\n"
            "3. Do not assign more than 4 categories per user.\n"
            "4. Display only the classification in the specified format. Do not include any additional text or explanation."
        )
        
        # Combine content and question
        prompt = [
            {"role": "system", "content": content},
            {"role": "user", "content": llm_input}
        ]
        
        # LLM initialization (adjust this part based on your actual LLM class and API)
        llm = Ollama(model="llama3.1")
        response = llm.invoke(prompt)
        
        # Step 3: Process the LLM response
        # Assuming the response is a string with classifications per user
        all_responses.append(response)  # Store the responses for each batch
        
        print(f"Batch {i // batch_size + 1} processed.")

    return all_responses

# Call the function to process the users in batches
all_classifications = classify_users_in_batches(usernames_with_data)

# Output the results (you can store or manipulate them as needed)
for batch_response in all_classifications:
    print(batch_response)

Batch 1 processed.
Batch 2 processed.
Batch 3 processed.
Batch 4 processed.
Batch 5 processed.
Batch 6 processed.
Batch 7 processed.
Batch 8 processed.
Batch 9 processed.
Batch 10 processed.
Batch 11 processed.
Batch 12 processed.
Batch 13 processed.
Batch 14 processed.
Batch 15 processed.
Batch 16 processed.
Batch 17 processed.
Batch 18 processed.
Batch 19 processed.
Batch 20 processed.
Batch 21 processed.
Batch 22 processed.
Batch 23 processed.
Batch 24 processed.
Batch 25 processed.
Batch 26 processed.
Batch 27 processed.
Batch 28 processed.
Batch 29 processed.
Batch 30 processed.
Batch 31 processed.
Batch 32 processed.
Batch 33 processed.
Batch 34 processed.
Batch 35 processed.
Batch 36 processed.
Batch 37 processed.
Batch 38 processed.
Batch 39 processed.
Batch 40 processed.
Batch 41 processed.
Batch 42 processed.
Batch 43 processed.
Batch 44 processed.
Batch 45 processed.
Batch 46 processed.
Batch 47 processed.
Batch 48 processed.
Batch 49 processed.
Batch 50 processed.
Batch 51 

In [43]:
with open('classifications.txt', 'w') as text_file:
    for batch_response in all_classifications:
        text_file.write(f"{batch_response}\n\n")

In [22]:
#process text file
import re

all_users_vector = list(usernames_with_data.keys())

# Path to your text file
file_path = 'classifications.txt'

# Initialize an empty dictionary to store usernames and categories
user_categories = {}

# Read the text file
with open(file_path, "r") as file:
    for line in file:
        # Remove leading/trailing whitespace
        line = line.strip()
        if not line:
            continue

        # Remove the "User:" prefix if it exists
        line = re.sub(r"^User:", "", line)
        line = re.sub(r"^Human:", "", line)

        # Split the line only at the first colon to get the username and categories
        if ":" in line:
            username, categories = line.split(":", 1)
            username = username.strip()
            categories = categories.strip().split(", ")

            # Check if the extracted username is in your list
            if username in all_users_vector:
                user_categories[username] = categories

# Print the resulting dictionary
print(user_categories)

#verification
missing_users = [user for user in all_users_vector if user not in user_categories]
print(len(missing_users))

# 2. Print the result
if not missing_users:
    print("All users are present in the user_categories dictionary.")
else:
    print(f"Missing users: {', '.join(missing_users)}")

{'!---slappdash---!': ['Entertainment', 'Music', 'Business', 'Sports'], '%D0%90': ['Education', 'Film', 'Geography', 'Media'], "'sed": ['Animation', 'Comedy', 'Media', 'Television'], '(.Y.)': ['Music', 'Lists', 'People', 'Sports'], '(aeropagitica)': ['Media', 'Entertainment', 'Culture', 'Science'], '*Kat*': ['Health', 'Education', 'Society', 'Government'], '*drew': ['Geography', 'Film', 'Media'], '-Midorihana-': ['Education', 'Technology', 'Community', 'Events'], '-Ril-': ['Mythology', 'Religion', 'Culture', 'Science'], '123Hedgehog456': ['Entertainment', 'History', 'People', 'Sports'], '125.162.150.88': ['Military', 'Politics', 'Society', 'Time'], '131.211.210.14': ['Business', 'Communication', 'Economy', 'Geography'], '131.211.210.157': ['Culture', 'Entertainment', 'History', 'Technology'], '156.34.142.110': ['Music', 'Concepts'], '172': ['Government', 'History', 'Philosophy', 'Society'], '17Drew': ['Music', 'People', 'Entertainment', 'Culture'], '194.6.242.2': ['Education', 'Informa

In [ ]:

# Function to clean the dictionary
def clean_dict(user_categories):
    cleaned_dict = {}
    for username, categories in user_categories.items():
        # Remove NaN, None, and empty strings from the categories list
        cleaned_categories = [
            category for category in categories if category and category not in [None, np.nan, "NaN", ""]
        ]

        # Only add the entry if there are valid categories
        if cleaned_categories:
            cleaned_dict[username] = cleaned_categories

    return cleaned_dict

# Clean the dictionary
user_categories = clean_dict(user_categories)
print('user with categories:',len(user_categories))
print('total user:',len(all_users_vector))


user with categories: 7357
total user: 7476


In [ ]:
# Save the cleaned dictionary to a JSON file
output_filename = "user_categories.json"
with open(output_filename, "w") as json_file:
    json.dump(user_categories, json_file, indent=4)